In [17]:
def svd_recommend(rating_matrix,movieId_vs_tmdbId,top_k_products=10):
    nb_users = rating_matrix.shape[0]
    nb_products = rating_matrix.shape[1]

    # Create a Tensorflow graph
    graph = tf.Graph()

    with graph.as_default():
        user_item_matrix = tf.placeholder(tf.float32, shape=(nb_users, nb_products))

        # SVD
        S, U, V = tf.svd(user_item_matrix)

        S = tf.diag(S)        
    
        Su = tf.matmul(U, tf.sqrt(S))
        Si = tf.matmul(tf.sqrt(S), tf.transpose(V))

        # Compute user ratings
        ratings_t = tf.matmul(Su, Si)
        
        # Pick top k suggestions
        best_ratings_t, best_items_t = tf.nn.top_k(ratings_t, top_k_products)

    # Create Tensorflow session
    session = tf.InteractiveSession(graph=graph)

    # Compute the top k suggestions for all users
    

    best_items,best_items_rating = session.run([best_items_t,best_ratings_t],feed_dict = {
        user_item_matrix: rating_matrix
    })

    # convert to tmdb id
    best_items = np.array(best_items)
    
    ans_items = []
    for row in best_items:
        tmp = []
        for item in row:
            try:
                tmp.append(movieId_vs_tmdbId[str(item)])
            except:
                continue

        ans_items.append(tmp)
    # return item are TMDB ID and corresponding predictive rating
    return ans_items,best_items_rating



In [ ]:
def cosine_similarity_matrix(data):
    '''
    calculate cosine similarity for user & item
    
    default: user is in row, item in column
    ''' 
    dim = data.shape
    
    constant = tf.constant(1e-9,dtype=tf.float32)

    df = tf.placeholder(shape=[dim[0],dim[1]],dtype=tf.float32)

    similar_user = tf.matmul(df,tf.transpose(df)) + constant

    norm_user = tf.reshape(tf.sqrt(tf.diag_part(similar_user)),[-1,1])

    norm_user_matrix = tf.matmul(norm_user,tf.transpose(norm_user))

    similar_user = similar_user/norm_user_matrix


    with tf.Session() as sess:
        ans = sess.run(similar_user,feed_dict={df:data})

    return ans


In [ ]:
def prediction_(similarity_matrix,rating_matrix):
    '''default we think df row is user, column is id'''
    similarity_matrix = tf.convert_to_tensor(similarity_matrix,dtype=tf.float32)
    rating_matrix = tf.convert_to_tensor(rating_matrix,dtype=tf.float32)
    
    numerator = tf.matmul(similarity_matrix,rating_matrix)
    
    denominator = tf.reduce_sum(tf.abs(similarity_matrix),axis=0)
    denominator = tf.reshape(denominator,shape=[-1,1])

    ans = numerator/denominator
    
    with tf.Session() as sess:
        return sess.run(ans)
    

def error(prediction,real_value):
    
    prediction = tf.convert_to_tensor(prediction)
    real_value = tf.convert_to_tensor(real_value)
    
    ans = tf.losses.mean_squared_error(predictions=prediction,labels=real_value)
    with tf.Session() as sess:
        return sess.run(ans)

In [ ]:
api_key = 'api_key=b1ab9a6e9622fe27f6dd2a81065224fc'
# this function is used to get poster link 
def get_poster(TMDB_id,api_key=api_key):
    
    url_full = 'https://api.themoviedb.org/3/movie/'+ str(TMDB_id)+'?'+str(api_key)
    url_content = requests.get(url_full)
    url_content = url_content.text
    post_ip = json.loads(url_content)['poster_path']
    post_ip_complete = 'https://image.tmdb.org/t/p/w500'+ post_ip
    
    return post_ip_complete

# this function is used to display images
def display_images(url_list):
    n_display = len(url_list)
    images = ''
    for i in range(n_display):
        images += "<img style='width: 100px; margin: 0px; \
                float: left; border: 1px solid black;' src='%s' />" \
                % url_list[i]

    display(HTML(images)) 
    
    

# this function is used for deep learning part
def similar_select(similarity_matrix,movieId_vs_tmdbId='',n=5,target_id=0):

    movies = [ movieId_vs_tmdbId[str(x)] \
          for x in np.argsort(similarity_matrix[target_id,:])[:-n-1:-1] ] 

    return movies





In [ ]:
def show_result(similarity_matrix,movieId_vs_tmdbId,movie_list,target_id = 0,n = 5):
    
    movie_selected_id = [x for x in np.argsort(similarity_movie[target_id,:])[-n-1:]]

    movie_selected = [movie_list[i] for i in movie_selected_id]

    url_list = [get_poster(x) for x in movie_selected]
    get_poster(movie_selected[0])
    display_images(url_list)

